In [1]:
from keras.applications import VGG16

Using TensorFlow backend.


In [2]:
base_model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (224, 224, 3))
base_model.layers[0].__class__.__name__
base_model.layers[0].input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [3]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
for layer in base_model.layers:
    layer.trainable=False
for (i,layer) in enumerate(base_model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [6]:
from keras.layers import Dense, Flatten
from keras.models import Sequential

top_model = base_model.output
top_model = Flatten()(top_model)
top_model = Dense(512, activation='relu')(top_model)   
top_model = Dense(256, activation='relu')(top_model)   
top_model = Dense(128, activation='relu')(top_model) 
top_model = Dense(2, activation='softmax')(top_model)  
top_model

<tf.Tensor 'dense_4/Softmax:0' shape=(None, 2) dtype=float32>

In [7]:
from keras.models import Model
model = Model(inputs=base_model.input, outputs=top_model)

In [8]:
model.output

<tf.Tensor 'dense_4/Softmax:0' shape=(None, 2) dtype=float32>

In [9]:
model.layers

In [10]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [11]:
from keras.preprocessing.image import ImageDataGenerator

In [12]:
train_datagen = ImageDataGenerator(
    
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [14]:
train_data="C:/Users/Dell/Desktop/maskproject/trainingphotos"
train_generator = train_datagen.flow_from_directory(
        train_data,
        target_size=(224, 224),
        class_mode='categorical')

Found 400 images belonging to 2 classes.


In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data="C:/Users/Dell/Desktop/maskproject/testingphotos"
test_generator = test_datagen.flow_from_directory(
        test_data,
        target_size=(224, 224),
        class_mode='categorical',
        shuffle=False)

Found 53 images belonging to 2 classes.


In [16]:
from keras.optimizers import RMSprop
model.compile(optimizer = RMSprop(lr=0.0001),
                 loss = 'categorical_crossentropy',
                 metrics =['accuracy']
                )

In [17]:
model.fit_generator(train_generator, epochs=2,steps_per_epoch=20, validation_data=test_generator,
                                validation_steps = 53)



Epoch 1/2
20/20 [==============================] - 615s 31s/step - loss: 0.1465 - accuracy: 0.9342 - val_loss: 9.6239e-04 - val_accuracy: 1.0000
Epoch 2/2
20/20 [==============================] - 599s 30s/step - loss: 6.2077e-04 - accuracy: 1.0000 - val_loss: 5.7548e-05 - val_accuracy: 1.0000


In [1]:
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array
from numpy import array, expand_dims


Using TensorFlow backend.


In [2]:
import cv2
model = load_model("vggsafe.h5")
testing_image = "C:/Users/Dell/Desktop/maskproject/testingphotos/RCM1/image0.jpg"
image = load_img(testing_image, target_size=(224, 224))
image.show(testing_image)

image = array(image)
image = expand_dims(image, axis=0)
if model.predict(image)[0][0] > 0.9:
    print("not wearing mask")
if model.predict(image)[0][1] > 0.9:
    print("wearing mask")

not wearing mask
